In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [2]:
import gradio as gr
import pandas as pd
import uuid

# Path to the CSV file
csv_file = "annotations.csv"

# Predefined languages and regions
languages = {
    "English": "en",
    "French": "fr",
    "Italian": "it",
    "Marathi": "mr",
    "Portuguese (Brazil)": "pt-BR",
    "German": "de",
    "Romanian": "ro",
    "Russian": "ru",
    "Dutch": "nl",
    "Chinese": "zh",
    "Arabic": "ar",
}

regions = {
    "United States": "USA",
    "France": "FRA",
    "Netherlands": "NLD",
    "India": "IND",
    "China": "CHN",
    "Hong Kong": "HKG",
    "Brazil": "BRA",
    "Italy": "ITA",
    "Germany": "DEU",
    "Romania": "ROU",
    "Algeria": "DZA",
    "Egypt": "EGY",
    "Libya": "LBY",
    "Morocco": "MAR",
    "Sudan": "SDN",
    "Tunisia": "TUN",
}

# Function to save annotation
def save_annotation(
    bias_type, stereotype_origin_langs, additional_origin_langs,
    stereotype_valid_langs, additional_valid_langs,
    stereotype_valid_regions, additional_regions, stereotyped_entity, type_,
    en_biased_sentences, en_expression, en_comments, created_by, creator_attributes_optional
):
    # Combine checkbox and textbox inputs
    combined_origin_langs = stereotype_origin_langs + additional_origin_langs.split(",") if additional_origin_langs else stereotype_origin_langs
    combined_valid_langs = stereotype_valid_langs + additional_valid_langs.split(",") if additional_valid_langs else stereotype_valid_langs
    combined_valid_regions = stereotype_valid_regions + additional_regions.split(",") if additional_regions else stereotype_valid_regions

    # Prepare the data row
    data = {
        "index": str(uuid.uuid4()),
        "subset": "_gradio",
        "bias_type": bias_type,
        "stereotype_origin_langs": combined_origin_langs,
        "stereotype_valid_langs": combined_valid_langs,
        "stereotype_valid_regions": combined_valid_regions,
        "stereotyped_entity": stereotyped_entity,
        "type": type_,
        "en_biased_sentences": en_biased_sentences,
        "en_expression": en_expression,
        "en_comments": en_comments,
        "created_by": created_by,
        "creator_attributes_optional": creator_attributes_optional,
    }

    # Save to CSV
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        df = pd.DataFrame(columns=data.keys())

    df = df.append(data, ignore_index=True)
    df.to_csv(csv_file, index=False)

    return "Annotation saved successfully!"

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("### Annotation Form")

    bias_type = gr.CheckboxGroup(label="Bias Type", choices=["gender", "race", "age", "other"])

    with gr.Row():
        stereotype_origin_langs = gr.CheckboxGroup(
            label="Stereotype Origin Languages",
            choices=[f"{name} ({code})" for name, code in languages.items()]
        )
        additional_origin_langs = gr.Textbox(
            label="Additional Origin Languages (CSV format, e.g., Spanish:es, Japanese:ja)"
        )

    with gr.Row():
        stereotype_valid_langs = gr.CheckboxGroup(
            label="Stereotype Valid Languages",
            choices=[f"{name} ({code})" for name, code in languages.items()]
        )
        additional_valid_langs = gr.Textbox(
            label="Additional Valid Languages (CSV format, e.g., Spanish:es, Japanese:ja)"
        )

    with gr.Row():
        stereotype_valid_regions = gr.CheckboxGroup(
            label="Stereotype Valid Regions",
            choices=[f"{name} ({code})" for name, code in regions.items()]
        )
        additional_regions = gr.Textbox(
            label="Additional Regions (CSV format, e.g., Mexico:MEX, Canada:CAN)"
        )

    stereotyped_entity = gr.Textbox(label="Stereotyped Entity")
    type_ = gr.Dropdown(label="Type", choices=["declaration", "question", "statement"])
    en_biased_sentences = gr.Textbox(label="English Biased Sentences", lines=3)
    en_expression = gr.Checkbox(label="Expression Validity")
    en_comments = gr.Textbox(label="English Comments (optional)", lines=2)

    with gr.Row():
        created_by = gr.Textbox(label="Created By")
        creator_attributes_optional = gr.Textbox(label="Creator Attributes (JSON format)")

    submit = gr.Button("Submit Annotation")
    submit.click(
        save_annotation,
        inputs=[
            bias_type, stereotype_origin_langs, additional_origin_langs,
            stereotype_valid_langs, additional_valid_langs,
            stereotype_valid_regions, additional_regions, stereotyped_entity,
            type_, en_biased_sentences, en_expression, en_comments,
            created_by, creator_attributes_optional
        ],
        outputs=[gr.Textbox(label="Status")]
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7f825d58ece6ec1c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
